In [ ]:
"""
This directly builds on 
scripts/241105_char_sp_rasters_snippets.ipynb

Here, setting up all raster plots related to grammar.

"""


In [ ]:
%load_ext autoreload
%autoreload 2

from neuralmonkey.classes.session import load_mult_session_helper
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline

In [ ]:
# load all sessions for Pancho on 221020

# animal = "Diego"
# date = 230913

animal = "Diego"
date = 240827

MS = load_mult_session_helper(date, animal)

##### Generate snippets

In [ ]:
# This is best, genreeates snippest for each session, thne concats into a single SP
from neuralmonkey.classes.snippets import Snippets, extraction_helper
from neuralmonkey.classes.snippets import load_and_concat_mult_snippets
which_level = "stroke"
PRE_DUR = -0.4
POST_DUR = 0.4
# PRE_DUR = -0.8
# POST_DUR = 0.8
EVENTS_KEEP = ["00_stroke"]
DEBUG = True
SP, _ = load_and_concat_mult_snippets(MS, which_level, EVENTS_KEEP, DEBUG = DEBUG, 
                                      prune_low_fr_sites=False, 
                                      PRE_DUR=PRE_DUR, POST_DUR=POST_DUR)


In [ ]:
# [OPTIONAL] Clean up, applying expt specific params and extract features
# NOTE: Only do this if you want to clean up data -- e.g, only successful trials.

from neuralmonkey.analyses.rsa import rsagood_questions_dict, rsagood_questions_params
# question = "CHAR_BASE_stroke"
question = "RULE_ANBMCK_STROKE"
q_params = rsagood_questions_dict(animal, date, question)[question]

D, list_features_extraction = SP.datasetbeh_preprocess_clean_by_expt(
    ANALY_VER=q_params["ANALY_VER"], vars_extract_append=q_params["effect_vars"],
    substrokes_plot_preprocess=False)


In [ ]:
DS = SP.datasetbeh_extract_dataset('datstrokes')
sorted(DS.Dat.keys())

In [ ]:
DS.Dat["datseg"][0]

In [ ]:
sorted(list_features_extraction)

In [ ]:
# Inspect the data. 
# Each row represents a single combination of:
# (trial, chan, event). To see that, inspect the output of 

display(SP.DfScalar)

from pythonlib.tools.pandastools import grouping_print_n_samples
grouping_print_n_samples(SP.DfScalar, ["trialcode", "chan", "event_aligned"])


# Cleaning beh data

# Plot rasters

In [ ]:
LIST_VAR = [
    ("chunk_rank", "chunk_within_rank_semantic"), # strongest test of index within chunk, and chunk index.
    # "chunk_within_rank_semantic", # Good, but slightly redudant with ("chunk_rank", "chunk_within_rank_semantic"). Consider activating

    # Encoding of shape even during gap?
    "CTXT_shape_prev", # var = 2-motifs, conditioned on the shape of 2nd stroke.

    # -- N in chunk (analysis)
    "chunk_n_in_chunk", # n at start or end of chunk [general across n_in_chunk]


    # These dont need to do chunk_within_rank_fromlast, since it would be very similar to this (chunk_within_rank)
    "chunk_within_rank_semantic", #

    # -- N in chunk (visualtions)
    "chunk_n_in_chunk", # subplots = indices within chunk (color by n in chunk)
    "chunk_n_in_chunk", # subplots = indices within chunk (color by n in chunk)
]

# ["chunk_rank", "chunk_within_rank", "chunk_within_rank_fromlast"], # NEW - effect of onset
LIST_VARS_OTHERS = [
    # This, checked closely by hand, as the minimum for getting very clean M1
    ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_locoffclust_prev", "shape", "loc_on_clust", "loc_off_clust", "CTXT_shape_prev", "CTXT_loconclust_next"],
    # ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_loc_prev", "shape", "gridloc", "CTXT_loc_next"],

    ("shape", "CTXT_loc_prev", "loc_on_clust", "loc_off_clust", "CTXT_loconclust_next", "CTXT_shape_next"), # Good -- tight control!

    # GOOD - 2 variations, 2nd is more controlled.
    ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_locoffclust_prev", "chunk_within_rank_semantic", "shape", "loc_on_clust", "CTXT_loc_next"],
    # ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_locoffclust_prev", "chunk_within_rank_semantic", "shape", "loc_on_clust", "CTXT_loc_next", "CTXT_shape_prev"],

    # GOOD - 4 variations, each with stronger control.
    # ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_locoffclust_prev", "shape", "loc_on_clust", "loc_off_clust"]
    # ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_locoffclust_prev", "shape", "loc_on_clust", "loc_off_clust", "CTXT_loconclust_next"]
    ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_locoffclust_prev", "shape", "loc_on_clust", "loc_off_clust", "CTXT_loconclust_next", "CTXT_shape_prev"],
    # ["stroke_index_is_first", "stroke_index_is_last_tskstks", "CTXT_locoffclust_prev", "shape", "loc_on_clust", "loc_off_clust", "CTXT_loconclust_next", "CTXT_shape_next"]

    ("chunk_rank", "shape", "chunk_within_rank"), # [vislauzations] NEW: coloring by n in chunk --> consistent across those?
    ("chunk_rank", "shape", "chunk_within_rank_fromlast"), # same, but align to last stroke in chunk.
]

# For some, should allow even if just 1 class.
LIST_OVERWRITE_lenient_n = [OVERWRITE_lenient_n for _ in range(len(LIST_VARS_OTHERS))]
LIST_OVERWRITE_lenient_n[4] = 1
LIST_OVERWRITE_lenient_n[5] = 1



In [ ]:
SP.sitegetter_map_region_to_sites("preSMA_a")
for chan in SP.sitegetter_map_region_to_sites("preSMA_p"):
    print(chan, " -- ", SP.session_sitegetter_summarytext(chan))

In [ ]:
# Quick testing
chan = 1348

# GOOD
# 2. subplots = (cr, shape, rankfromend); color = n_in_chunk
var = "chunk_n_in_chunk"
# vars_others = ["chunk_rank", "shape", "chunk_within_rank_fromlast"]
vars_others = ["shape", "chunk_within_rank_fromlast"]

# # 2. subplots = (cr, shape, rankfromend); color = n_in_chunk
# var = "chunk_n_in_chunk"
# # vars_others = ["chunk_rank", "shape", "chunk_within_rank_fromlast"]
# vars_others = ["chunk_rank", "shape", "chunk_within_rank"]

# GOOD
# 2. subplots = (cr, shape, rankfromend); color = n_in_chunk
var = "chunk_within_rank_fromlast"
# vars_others = ["chunk_rank", "shape", "chunk_n_in_chunk"]
vars_others = ["shape", "chunk_n_in_chunk"]

event = "00_stroke"
plotvers = ("smfr")
balance_same_levels_across_ovar = False
clean = False
fig, axesall = SP.plotgood_rasters_smfr_each_level_combined(chan, var, vars_others,
                                                            event=event,
                                                            plotvers=plotvers,
                                                            OVERWRITE_n_min=OVERWRITE_n_min,
                                                            OVERWRITE_lenient_n=OVERWRITE_lenient_n,
                                                            balance_same_levels_across_ovar=balance_same_levels_across_ovar,
                                                            clean=clean)


In [ ]:
# Plot using grid.

SP.DfScalar["dummy"] = "dummy"
# grid_subplots = ()

var = "gridloc"
# vars_others = ["chunk_rank", "shape", "chunk_within_rank_fromlast"]
vars_others = ["shape", "chunk_within_rank_fromlast"]

fig, axesall = SP.plotgood_smfr_each_level_subplot_grid_by_vars(chan, var, vars_others[0], vars_others[1], 
    PLOT_VER="smfr", event=None)


In [ ]:
# [TOok too long, so skipped] Trying to assign global chunk rank column
DS = SP.datasetbeh_extract_dataset("datstrokes")
DS.Dat["date"] = "dummy"
chunk_rank_global_extract(DS.Dat)
SP.
from pythonlib.dataset.dataset_analy.grammar import chunk_rank_global_extract
SP.DfScalar["date"] = date
chunk_rank_global_extract(SP.DfScalar)
SP.datasetbeh_append_column_mult(["chunk_rank_global", "crg_shape"], DS)
# a new column
from pythonlib.tools.pandastools import append_col_with_grp_index
SP.DfScalar = append_col_with_grp_index(SP.DfScalar, ["chunk_rank_global", "shape"], "crg_shape")

In [ ]:
n_min_trials_per_shape = 4
event = "00_stroke"
OVERWRITE_n_min = n_min_trials_per_shape
OVERWRITE_lenient_n = 1
PATH_ANALYSIS_OUTCOMES = "/tmp"
LIST_VAR = [
    "chunk_n_in_chunk",
    "chunk_within_rank_fromlast",
]
LIST_VARS_OTHERS = [
    ("shape", "chunk_within_rank_fromlast"),
    ("shape", "chunk_n_in_chunk"),
]

# For some, should allow even if just 1 class.
LIST_OVERWRITE_lenient_n = [OVERWRITE_lenient_n for _ in range(len(LIST_VARS_OTHERS))]

SAVEDIR = f"{PATH_ANALYSIS_OUTCOMES}/recordings/main/RASTERS/{animal}-{date}/{question}"
os.makedirs(SAVEDIR, exist_ok=True)

for var, vars_others, OVERWRITE_lenient_n in zip(LIST_VAR, LIST_VARS_OTHERS, LIST_OVERWRITE_lenient_n):
    for event in EVENTS_KEEP:
        plotter(SP, var, vars_others, event, SAVEDIR, OVERWRITE_n_min, OVERWRITE_lenient_n)

